# Calculate a cross-validated error of the GP model
---

# Import Modules

In [ ]:
import os
import sys

import pickle

import pandas as pd

import chart_studio.plotly as py
import plotly.graph_objs as go

sys.path.insert(0, os.path.join(os.environ["PROJ_irox"], "data"))
from proj_data_irox import (
    bulk_dft_data_path, unique_ids_path,
    df_features_pre_opt_path,
    df_features_post_opt_path)

In [ ]:
# workflow/ml_modelling/00_ml_workflow/190611_new_workflow/02_gaus_proc
sys.path.insert(0, 
    os.path.join(
        os.environ["PROJ_irox"],
        "workflow/ml_modelling/00_ml_workflow",
        "190611_new_workflow/02_gaus_proc",
        )
    )
from gp_methods import gp_model_catlearn

# Read Data

In [ ]:
path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/00_ml_workflow/190611_new_workflow",
    "02_gaus_proc/out_data",
    "data_dict.pickle")
with open(path_i, "rb") as fle:
    data = pickle.load(fle)

In [ ]:
path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/00_ml_workflow/190611_new_workflow",
    "02_gaus_proc/out_data",
    "train_x_y.pickle")
with open(path_i, "rb") as fle:
    train_x_y = pickle.load(fle)

train_x_y.keys()

mean_y = train_x_y["train_y"].mean()
std_y = train_x_y["train_y"].std()

std_y

In [ ]:
with open(bulk_dft_data_path, "rb") as fle:
    df_bulk_dft = pickle.load(fle)

with open(df_features_pre_opt_path, "rb") as fle:
    df_features_pre = pickle.load(fle)

with open(df_features_post_opt_path, "rb") as fle:
    df_features_post = pickle.load(fle)

df_ids = pd.read_csv(unique_ids_path)

In [ ]:
# TEMP
features_i = df_features_post.iloc[0:2].values

In [ ]:
last_ind = list(data.keys())[-1]

data_i = data[last_ind]
print("data_i.keys(): ", data_i.keys())

GP = data_i["gp_instance"]

# pred = GP.predict(
#     test_fp=features_i,
#     uncertainty=True,
#     )

In [ ]:
gp_model_catlearn

pred = GP.predict(
    test_fp=train_x_y["train_x"],
    uncertainty=True,
    )

pred["prediction"]

prediction = pd.DataFrame(pred, index=train_x_y["train_x"].index)


In [ ]:


# prediction.index.tolist()

In [ ]:

data = []
trace = go.Scatter(
    x=prediction.index.tolist(),
    y=std_y * prediction["prediction"] + mean_y,
    mode="markers",
    name="GP Prediction",
    )
data.append(trace)

trace = go.Scatter(
    x=prediction.index.tolist(),
    y=std_y * train_x_y["train_y_standard"] + mean_y,
    mode="markers",
    name="DFT Data",
    )
data.append(trace)


fig = go.Figure(data=data)
fig.show()